### Load dependencies

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2
import datetime
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

from tqdm.notebook import tqdm_notebook

### Load train images and labels

In [2]:
labels = pd.read_csv("eye_gender_data/Training_set.csv") # loading the labels
labels.head()

,filename,label
0,Image_1.jpg,male
1,Image_2.jpg,female
2,Image_3.jpg,female
3,Image_4.jpg,female
4,Image_5.jpg,male


In [3]:
file_paths = [[fname, 'eye_gender_data/train/' + fname] for fname in labels['filename']]

In [4]:
# Confirm if number of images is same as number of labels given
if len(labels) == len(file_paths):
    print('Number of labels i.e. ', len(labels), 'matches the number of filenames i.e. ', len(file_paths))
else:
    print('Number of labels does not match the number of filenames')

Number of labels i.e.  9220 matches the number of filenames i.e.  9220


In [5]:
images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
images.head()

,filename,filepaths
0,Image_1.jpg,eye_gender_data/train/Image_1.jpg
1,Image_2.jpg,eye_gender_data/train/Image_2.jpg
2,Image_3.jpg,eye_gender_data/train/Image_3.jpg
3,Image_4.jpg,eye_gender_data/train/Image_4.jpg
4,Image_5.jpg,eye_gender_data/train/Image_5.jpg


In [6]:
train_data = pd.merge(images, labels, how = 'inner', on = 'filename')
train_data.head()

,filename,filepaths,label
0,Image_1.jpg,eye_gender_data/train/Image_1.jpg,male
1,Image_2.jpg,eye_gender_data/train/Image_2.jpg,female
2,Image_3.jpg,eye_gender_data/train/Image_3.jpg,female
3,Image_4.jpg,eye_gender_data/train/Image_4.jpg,female
4,Image_5.jpg,eye_gender_data/train/Image_5.jpg,male


In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_data['label'] = le.fit_transform(train_data['label'])

In [8]:
data = [] # initialize an empty numpy array
image_size = 53
for i in range(len(train_data)):

    img_array = cv2.imread(train_data['filepaths'][i], cv2.IMREAD_GRAYSCALE) # converting the image to gray scale
    new_img_array = cv2.resize(img_array, (image_size, image_size)) # resizing the image array
    data.append([new_img_array, train_data['label'][i]])

In [9]:
x = []
y = []
for image in data:
    x.append(image[0])
    y.append(image[1])

# converting x & y to numpy array as they are list
train_images = np.array(x)
train_labels = np.array(y)

In [10]:
np.unique(y, return_counts=True)

(array([0, 1]), array([4162, 5058], dtype=int64))

In [11]:
# shape of an image
train_images[0].shape

(53, 53)

In [12]:
train_images.shape

(9220, 53, 53)

In [13]:
# reshape for feeding into the model
train_images_gr = train_images.reshape(train_images.shape[0], 53, 53, 1)

print('\nTrain_images.shape: {}, of {}'.format(train_images_gr.shape, train_images_gr.dtype))


Train_images.shape: (9220, 53, 53, 1), of uint8


In [14]:
# normalize values
train_images = train_images / 255.0

### Building the model

In [15]:
cnn = tf.keras.models.Sequential([
tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3),  strides=(1, 1), activation='relu', padding='valid', input_shape=(53, 53, 1)),
tf.keras.layers.MaxPooling2D((2, 2)),

tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3),  strides=(1, 1), activation='relu', padding='valid', input_shape=(53, 53, 1)),
tf.keras.layers.MaxPooling2D((2, 2)),

tf.keras.layers.Flatten(),
    
tf.keras.layers.Dense(128, activation='relu'),
tf.keras.layers.Dropout(rate=0.3),
    
tf.keras.layers.Dense(2, activation='sigmoid')
])

In [16]:
cnn.compile(optimizer='adam',
loss='sparse_categorical_crossentropy',
metrics=['accuracy'])

earlystopping_callback = EarlyStopping(
    monitor='val_loss', min_delta=0, patience=2, 
    verbose=1, restore_best_weights=True, mode='auto'
)

# view model layers
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 51, 51, 16)        160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 25, 25, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 23, 23, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 11, 11, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 3872)              0         
                                                                 
 dense (Dense)               (None, 128)               4

In [ ]:
history = cnn.fit(x=train_images, y=train_labels, epochs=50, batch_size=64, validation_split=0.2, callbacks=[earlystopping_callback])

Epoch 1/50
116/116 [==============================] - 11s 94ms/step - loss: 0.5895 - accuracy: 0.6870 - val_loss: 0.4857 - val_accuracy: 0.7782
Epoch 2/50
116/116 [==============================] - 11s 92ms/step - loss: 0.4345 - accuracy: 0.8053 - val_loss: 0.3668 - val_accuracy: 0.8476
Epoch 3/50
116/116 [==============================] - 11s 98ms/step - loss: 0.3738 - accuracy: 0.8416 - val_loss: 0.3267 - val_accuracy: 0.8666
Epoch 4/50
116/116 [==============================] - 11s 97ms/step - loss: 0.3328 - accuracy: 0.8625 - val_loss: 0.3087 - val_accuracy: 0.8747
Epoch 5/50
116/116 [==============================] - 11s 98ms/step - loss: 0.3153 - accuracy: 0.8715 - val_loss: 0.2847 - val_accuracy: 0.8883
Epoch 6/50
116/116 [==============================] - 11s 96ms/step - loss: 0.2848 - accuracy: 0.8826 - val_loss: 0.2742 - val_accuracy: 0.8883
Epoch 7/50
116/116 [==============================] - 11s 94ms/step - loss: 0.2689 - accuracy: 0.8879 - val_loss: 0.2851 - val_accuracy:

### Plot learning curves

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(22, 8))

history_df = pd.DataFrame(history.history)
history_df[['loss', 'val_loss']].plot(kind='line', ax=ax[0])
history_df[['accuracy', 'val_accuracy']].plot(kind='line', ax=ax[1]);

### Make predictions

In [ ]:
test_labels = pd.read_csv("eye_gender_data/Testing_set.csv")

In [ ]:
file_paths = [[fname, 'eye_gender_data/test/' + fname] for fname in test_labels['filename']]

In [ ]:
# Confirm if number of images is same as number of labels given
if len(test_labels) == len(file_paths):
    print('Number of labels i.e. ', len(test_labels), 'matches the number of filenames i.e. ', len(file_paths))
else:
    print('Number of labels does not match the number of filenames')

In [ ]:
test_data = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
test_data.head()

In [ ]:
data = []
image_size = 53
for i in range(len(test_data)):

    img_array = cv2.imread(test_data['filepaths'][i], cv2.IMREAD_GRAYSCALE) # converting the image to gray scale
    new_img_array = cv2.resize(img_array, (image_size, image_size)) # resizing the image array
    data.append([new_img_array])

In [ ]:
x = []
for image in data:
    x.append(image[0])

# converting x & y to numpy array as they are lists
test_images = np.array(x)

In [ ]:
test_images_gr = test_images.reshape(test_images.shape[0], 53, 53, 1)

In [ ]:
test_images = test_images / 255.0

In [ ]:
predictions = cnn.predict(test_images)

In [ ]:
prediction_labels = np.argmax(predictions, axis=1)

In [ ]:
results = pd.DataFrame({'label': prediction_labels})

In [ ]:
replacement_dict = {1 : 'male', 0 : 'female'}
results = results.replace({'label' : replacement_dict})

results.to_csv('predictions.csv', index = False)